In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
import collections
%matplotlib inline

from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.graphics.gofplots import qqplot

# Raw DataFrame

In [2]:
# read in the Chicagov Divvy Bicycle Sharing Data csv files from Kaggle
pd.set_option('display.max_columns', 30)
filename = 'data.csv'
df = pd.read_csv(filename)
df.head()

,trip_id,year,month,week,day,hour,usertype,gender,starttime,stoptime,tripduration,temperature,events,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
0,2355134,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:57:00,2014-07-01 00:07:00,10.066667,68.0,tstorms,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,303,Broadway & Cornelia Ave,41.945512,-87.645980,15.0
1,2355133,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:56:00,2014-07-01 00:00:00,4.383333,68.0,tstorms,282,Halsted St & Maxwell St,41.864580,-87.646930,15.0,22,May St & Taylor St,41.869482,-87.655486,15.0
2,2355130,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:33:00,2014-06-30 23:35:00,2.100000,68.0,tstorms,327,Sheffield Ave & Webster Ave,41.921687,-87.653714,19.0,225,Halsted St & Dickens Ave,41.919936,-87.648830,15.0
3,2355129,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:26:00,2014-07-01 00:24:00,58.016667,68.0,tstorms,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0,194,State St & Wacker Dr,41.887155,-87.627750,11.0
4,2355128,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:16:00,2014-06-30 23:26:00,10.633333,68.0,tstorms,320,Loomis St & Lexington St,41.872187,-87.661501,15.0,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0


# Data Cleaning and Wrangling

### Trip ID Column

In [3]:
# drop rows with duplicate data 
data = df.copy()
data = data.drop_duplicates().sort_values('trip_id')

### General cleaning steps for easier use

In [4]:
# convert start and stop times to datetimes
data['starttime'] = pd.to_datetime(data['starttime'])
data['stoptime'] = pd.to_datetime(data['stoptime'])

In [5]:
# rename day column to numerical day of week
data = data.rename(columns={'day':'num_day_of_week'})

In [6]:
# create new column with the name of the week 
data['day_of_week'] = data['starttime'].dt.weekday_name

In [7]:
# create new column with numerical day of the month
data['day'] = data['starttime'].dt.day

In [8]:
# sort columns
pd.set_option('display.max_columns', 30)
data = data[['trip_id','year','month','week','day','hour','num_day_of_week','day_of_week',
             'starttime','stoptime','tripduration','temperature','events','usertype','gender',
             'from_station_id','from_station_name','latitude_start','longitude_start','dpcapacity_start',
             'to_station_id','to_station_name','latitude_end','longitude_end','dpcapacity_end']]
data.head()

,trip_id,year,month,week,day,hour,num_day_of_week,day_of_week,starttime,stoptime,tripduration,temperature,events,usertype,gender,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
576426,1109427,2014,1,1,1,1,2,Wednesday,2014-01-01 01:12:00,2014-01-01 01:18:00,5.766667,10.9,not clear,Subscriber,Male,240,Sheridan Rd & Irving Park Rd,41.954245,-87.654406,23.0,245,Clarendon Ave & Junior Ter,41.961004,-87.649603,15.0
576425,1109431,2014,1,1,1,1,2,Wednesday,2014-01-01 01:43:00,2014-01-01 01:53:00,10.833333,10.9,not clear,Subscriber,Male,113,Bissell St & Armitage Ave,41.918440,-87.652220,15.0,94,Clark St & Armitage Ave,41.918306,-87.636282,19.0
576424,1109432,2014,1,1,1,1,2,Wednesday,2014-01-01 01:43:00,2014-01-01 01:53:00,10.866667,10.9,not clear,Subscriber,Male,113,Bissell St & Armitage Ave,41.918440,-87.652220,15.0,94,Clark St & Armitage Ave,41.918306,-87.636282,19.0
576423,1109434,2014,1,1,1,1,2,Wednesday,2014-01-01 01:44:00,2014-01-01 01:59:00,15.266667,10.9,not clear,Subscriber,Male,50,Clark St & Congress Pkwy,41.875933,-87.630585,27.0,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0
576422,1109435,2014,1,1,1,1,2,Wednesday,2014-01-01 01:45:00,2014-01-01 02:00:00,15.066667,10.9,not clear,Subscriber,Female,50,Clark St & Congress Pkwy,41.875933,-87.630585,27.0,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0


### Capacity

In [9]:
departures = data[['starttime','from_station_name','trip_id','num_day_of_week']].sort_values(['from_station_name','starttime'])
departures = departures.rename(columns={'starttime':'datetime','from_station_name':'station_name'})

In [10]:
departures.head()

,datetime,station_name,trip_id,num_day_of_week
2146213,2015-05-17 14:06:00,2112 W Peterson Ave,5199604,6
2140518,2015-05-18 10:47:00,2112 W Peterson Ave,5213095,0
2106692,2015-05-22 17:29:00,2112 W Peterson Ave,5276252,4
2078846,2015-05-27 15:53:00,2112 W Peterson Ave,5356250,2
2074568,2015-05-27 18:57:00,2112 W Peterson Ave,5361887,2


In [11]:
# each departure is given a 1 value
departures['trip_counts'] = 1

In [12]:
arrivals = data[['starttime','to_station_name','trip_id','num_day_of_week']].sort_values(['to_station_name','starttime'])
arrivals = arrivals.rename(columns={'starttime':'datetime','to_station_name':'station_name'})

In [13]:
# each arrivals is given a -1 value
arrivals['trip_counts'] = -1

In [14]:
departures_and_arrivals = pd.concat([departures, arrivals])
departures_and_arrivals = departures_and_arrivals.sort_values(['station_name','datetime','num_day_of_week'])
departures_and_arrivals['date'] = departures_and_arrivals['datetime'].dt.date
departures_and_arrivals = departures_and_arrivals[['datetime','date','num_day_of_week','station_name','trip_id','trip_counts']]
departures_and_arrivals.head()

,datetime,date,num_day_of_week,station_name,trip_id,trip_counts
2146845,2015-05-17 12:41:00,2015-05-17,6,2112 W Peterson Ave,5197801,-1
2146213,2015-05-17 14:06:00,2015-05-17,6,2112 W Peterson Ave,5199604,1
2140550,2015-05-18 10:36:00,2015-05-18,0,2112 W Peterson Ave,5212990,-1
2140518,2015-05-18 10:47:00,2015-05-18,0,2112 W Peterson Ave,5213095,1
2107385,2015-05-22 16:41:00,2015-05-22,4,2112 W Peterson Ave,5274966,-1


In [15]:
# get running sum of departures-arrivals
departures_and_arrivals['cumulative_sum'] = departures_and_arrivals.groupby(['station_name','date','num_day_of_week'])['trip_counts'].cumsum()
departures_and_arrivals.head()


,datetime,date,num_day_of_week,station_name,trip_id,trip_counts,cumulative_sum
2146845,2015-05-17 12:41:00,2015-05-17,6,2112 W Peterson Ave,5197801,-1,-1
2146213,2015-05-17 14:06:00,2015-05-17,6,2112 W Peterson Ave,5199604,1,0
2140550,2015-05-18 10:36:00,2015-05-18,0,2112 W Peterson Ave,5212990,-1,-1
2140518,2015-05-18 10:47:00,2015-05-18,0,2112 W Peterson Ave,5213095,1,0
2107385,2015-05-22 16:41:00,2015-05-22,4,2112 W Peterson Ave,5274966,-1,-1


In [16]:
# maximum departures-arrivals for each station 
max_sum = departures_and_arrivals.groupby('station_name')['cumulative_sum'].max().reset_index()
max_sum.head()

,station_name,cumulative_sum
0,2112 W Peterson Ave,5
1,63rd St Beach,4
2,900 W Harrison,16
3,900 W Harrison St,15
4,Aberdeen St & Jackson Blvd,16


In [17]:
# maximum capacity at each station
max_cap = data[['from_station_name','dpcapacity_start']].drop_duplicates().groupby('from_station_name').max()
max_cap.head()

,dpcapacity_start
from_station_name,
2112 W Peterson Ave,15.0
63rd St Beach,23.0
900 W Harrison,19.0
900 W Harrison St,19.0
Aberdeen St & Jackson Blvd,15.0


In [18]:
df_merge_all = pd.merge(departures_and_arrivals, max_cap, how='left', left_on='station_name',right_on='from_station_name')


In [19]:
df_merge_all = df_merge_all.groupby(['station_name','date'])[['cumulative_sum','dpcapacity_start']].max().reset_index()
df_merge_all.head()

,station_name,date,cumulative_sum,dpcapacity_start
0,2112 W Peterson Ave,2015-05-17,0,15.0
1,2112 W Peterson Ave,2015-05-18,0,15.0
2,2112 W Peterson Ave,2015-05-22,0,15.0
3,2112 W Peterson Ave,2015-05-23,-1,15.0
4,2112 W Peterson Ave,2015-05-25,-1,15.0


In [20]:
df_merge_all['difference'] = df_merge_all['dpcapacity_start'] - df_merge_all['cumulative_sum']

In [21]:
# show all stations that exceed capacity
df_negative = df_merge_all[df_merge_all['difference']<-1]
df_negative.head()

,station_name,date,cumulative_sum,dpcapacity_start,difference
4320,Aberdeen St & Madison St,2014-09-06,32,19.0,-13.0
4321,Aberdeen St & Madison St,2014-09-07,23,19.0,-4.0
4322,Aberdeen St & Madison St,2014-09-08,24,19.0,-5.0
4330,Aberdeen St & Madison St,2014-09-17,39,19.0,-20.0
4331,Aberdeen St & Madison St,2014-09-18,42,19.0,-23.0


6205 out of 576538 unique stations and dates exceed capacity.

In [22]:
#data_rm_neg = data.copy

In [23]:
#data_rm_neg['date'] = data_rm_neg['starttime'].dt.date

In [24]:
#data_rm_neg = pd.merge(data_rm_neg, df_negative, how='outer', on='date', indicator=True).query('_merge == "left_only"').drop('_merge', 1)
#data_rm_neg.head()

## Departure Joined Dataframe

In [25]:
dfmodel = data.copy()

In [26]:
dfmodel = pd.get_dummies(dfmodel, columns = ['events','usertype','gender'])


In [27]:
dfmodel.head()

,trip_id,year,month,week,day,hour,num_day_of_week,day_of_week,starttime,stoptime,tripduration,temperature,from_station_id,from_station_name,latitude_start,...,to_station_name,latitude_end,longitude_end,dpcapacity_end,events_clear,events_cloudy,events_not clear,events_rain or snow,events_tstorms,events_unknown,usertype_Customer,usertype_Dependent,usertype_Subscriber,gender_Female,gender_Male
576426,1109427,2014,1,1,1,1,2,Wednesday,2014-01-01 01:12:00,2014-01-01 01:18:00,5.766667,10.9,240,Sheridan Rd & Irving Park Rd,41.954245,...,Clarendon Ave & Junior Ter,41.961004,-87.649603,15.0,0,0,1,0,0,0,0,0,1,0,1
576425,1109431,2014,1,1,1,1,2,Wednesday,2014-01-01 01:43:00,2014-01-01 01:53:00,10.833333,10.9,113,Bissell St & Armitage Ave,41.918440,...,Clark St & Armitage Ave,41.918306,-87.636282,19.0,0,0,1,0,0,0,0,0,1,0,1
576424,1109432,2014,1,1,1,1,2,Wednesday,2014-01-01 01:43:00,2014-01-01 01:53:00,10.866667,10.9,113,Bissell St & Armitage Ave,41.918440,...,Clark St & Armitage Ave,41.918306,-87.636282,19.0,0,0,1,0,0,0,0,0,1,0,1
576423,1109434,2014,1,1,1,1,2,Wednesday,2014-01-01 01:44:00,2014-01-01 01:59:00,15.266667,10.9,50,Clark St & Congress Pkwy,41.875933,...,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0,0,0,1,0,0,0,0,0,1,0,1
576422,1109435,2014,1,1,1,1,2,Wednesday,2014-01-01 01:45:00,2014-01-01 02:00:00,15.066667,10.9,50,Clark St & Congress Pkwy,41.875933,...,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0,0,0,1,0,0,0,0,0,1,1,0


In [28]:
dfdmodel = dfmodel.groupby(['from_station_name','year','month','day','hour','num_day_of_week']).agg({'tripduration':'mean',
                                                                        'temperature':'mean'}).reset_index()
dfdmodel.head()

,from_station_name,year,month,day,hour,num_day_of_week,tripduration,temperature
0,2112 W Peterson Ave,2015,5,17,14,6,16.550000,81.0
1,2112 W Peterson Ave,2015,5,18,10,0,4.166667,75.0
2,2112 W Peterson Ave,2015,5,22,17,4,13.616667,60.1
3,2112 W Peterson Ave,2015,5,27,15,2,14.300000,71.1
4,2112 W Peterson Ave,2015,5,27,18,2,15.450000,72.0


In [29]:
dfdmodel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4061061 entries, 0 to 4061060
Data columns (total 8 columns):
from_station_name    object
year                 int64
month                int64
day                  int64
hour                 int64
num_day_of_week      int64
tripduration         float64
temperature          float64
dtypes: float64(2), int64(5), object(1)
memory usage: 247.9+ MB


#### event percentages

In [30]:
dfdevents = dfmodel.copy()
dfdevents = dfdevents[['from_station_name','year','month','day','hour','num_day_of_week','events_cloudy','events_clear','events_not clear','events_rain or snow','events_tstorms','events_unknown']]
dfdevents.head()

,from_station_name,year,month,day,hour,num_day_of_week,events_cloudy,events_clear,events_not clear,events_rain or snow,events_tstorms,events_unknown
576426,Sheridan Rd & Irving Park Rd,2014,1,1,1,2,0,0,1,0,0,0
576425,Bissell St & Armitage Ave,2014,1,1,1,2,0,0,1,0,0,0
576424,Bissell St & Armitage Ave,2014,1,1,1,2,0,0,1,0,0,0
576423,Clark St & Congress Pkwy,2014,1,1,1,2,0,0,1,0,0,0
576422,Clark St & Congress Pkwy,2014,1,1,1,2,0,0,1,0,0,0


In [31]:
dfdevents['sum'] = (dfdevents['events_cloudy'] +
                   dfdevents['events_clear'] +
                   dfdevents['events_not clear'] +
                   dfdevents['events_rain or snow'] + 
                   dfdevents['events_tstorms'] +
                   dfdevents['events_unknown'])

In [32]:
dfdevents = dfdevents.sort_values(['from_station_name','year','month','day','hour','num_day_of_week'])

In [33]:
dfdevents = dfdevents.groupby(['from_station_name','year','month','day','hour','num_day_of_week']).sum().reset_index()
dfdevents.head();

In [34]:
dfdevents['events_cloudy_pct'] = (dfdevents['events_cloudy'] / dfdevents['sum'])*100
dfdevents['events_clear_pct'] = (dfdevents['events_clear'] / dfdevents['sum'])*100
dfdevents['events_not clear_pct'] = (dfdevents['events_not clear'] / dfdevents['sum'])*100
dfdevents['events_rain or snow_pct'] = (dfdevents['events_rain or snow'] / dfdevents['sum'])*100
dfdevents['events_tstorms_pct'] = (dfdevents['events_tstorms'] / dfdevents['sum'])*100
dfdevents['events_unknown_pct'] = (dfdevents['events_unknown'] / dfdevents['sum'])*100

In [35]:
dfdevents_pct = dfdevents[['from_station_name','year','month','day','hour','num_day_of_week','events_cloudy_pct','events_clear_pct','events_not clear_pct','events_rain or snow_pct','events_tstorms_pct','events_unknown_pct']]
dfdevents_pct.head();

#### usertype percentage

In [36]:
dfdusertype = dfmodel.copy()
dfdusertype = dfdusertype[['from_station_name','year','month','day','hour','num_day_of_week','usertype_Customer','usertype_Dependent','usertype_Subscriber']]
dfdusertype.head();

In [37]:
dfdusertype['sum'] = (dfdusertype['usertype_Customer'] +
                   dfdusertype['usertype_Dependent'] +
                   dfdusertype['usertype_Subscriber'])

In [38]:
dfdusertype = dfdusertype.sort_values(['from_station_name','year','month','day','hour','num_day_of_week'])

In [39]:
dfdusertype = dfdusertype.groupby(['from_station_name','year','month','day','hour','num_day_of_week']).sum().reset_index()
dfdusertype.head();

In [40]:
dfdusertype['usertype_Customer_pct'] = (dfdusertype['usertype_Customer'] / dfdusertype['sum'])*100
dfdusertype['usertype_Dependent_pct'] = (dfdusertype['usertype_Dependent'] / dfdusertype['sum'])*100
dfdusertype['usertype_Subscriber_pct'] = (dfdusertype['usertype_Subscriber'] / dfdusertype['sum'])*100

In [41]:
dfdusertype_pct = dfdusertype[['from_station_name','year','month','day','hour','num_day_of_week','usertype_Customer_pct','usertype_Dependent_pct','usertype_Subscriber_pct']]
dfdusertype_pct.head();

#### gender percentages

In [42]:
dfdgender = dfmodel.copy()
dfdgender = dfdgender[['from_station_name','year','month','day','hour','num_day_of_week','gender_Female','gender_Male']]
dfdgender.head();

In [43]:
dfdgender['sum'] = (dfdgender['gender_Female'] + dfdgender['gender_Male'])

In [44]:
dfdgender = dfdgender.sort_values(['from_station_name','year','month','day','hour','num_day_of_week'])

In [45]:
dfdgender = dfdgender.groupby(['from_station_name','year','month','day','hour','num_day_of_week']).sum().reset_index()
dfdgender.head();

In [46]:
dfdgender['gender_Female_pct'] = (dfdgender['gender_Female'] / dfdgender['sum'])*100
dfdgender['gender_Male_pct'] = (dfdgender['gender_Male'] / dfdgender['sum'])*100

In [47]:
dfdgender_pct = dfdgender[['from_station_name','year','month','day','hour','num_day_of_week','gender_Female_pct','gender_Male_pct','sum']]
dfdgender_pct.head();

### Join

In [48]:
joindf = pd.merge(dfdmodel, dfdevents_pct, how='left',on=['from_station_name','year','month','day','hour','num_day_of_week'])

In [49]:
joindf = pd.merge(joindf, dfdusertype_pct, how='left', on=['from_station_name','year','month','day','hour','num_day_of_week'])

In [50]:
departure_joindf = pd.merge(joindf, dfdgender_pct, how='left', on=['from_station_name','year','month','day','hour','num_day_of_week'])

In [51]:
departure_joindf = departure_joindf.rename(columns={'sum':'trip_count'})

In [52]:
departure_joindf['year'] = departure_joindf['year'].astype(str)
departure_joindf['month'] = departure_joindf['month'].astype(str)
departure_joindf['day'] = departure_joindf['day'].astype(str)

In [53]:
departure_joindf = departure_joindf.set_index('from_station_name')

In [54]:
departure_joindf.head()

,year,month,day,hour,num_day_of_week,tripduration,temperature,events_cloudy_pct,events_clear_pct,events_not clear_pct,events_rain or snow_pct,events_tstorms_pct,events_unknown_pct,usertype_Customer_pct,usertype_Dependent_pct,usertype_Subscriber_pct,gender_Female_pct,gender_Male_pct,trip_count
from_station_name,,,,,,,,,,,,,,,,,,,
2112 W Peterson Ave,2015,5,17,14,6,16.550000,81.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,1
2112 W Peterson Ave,2015,5,18,10,0,4.166667,75.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,1
2112 W Peterson Ave,2015,5,22,17,4,13.616667,60.1,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,1
2112 W Peterson Ave,2015,5,27,15,2,14.300000,71.1,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,1
2112 W Peterson Ave,2015,5,27,18,2,15.450000,72.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,1


In [55]:
departure_joindf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4061061 entries, 2112 W Peterson Ave to Yates Blvd & 75th St
Data columns (total 19 columns):
year                       object
month                      object
day                        object
hour                       int64
num_day_of_week            int64
tripduration               float64
temperature                float64
events_cloudy_pct          float64
events_clear_pct           float64
events_not clear_pct       float64
events_rain or snow_pct    float64
events_tstorms_pct         float64
events_unknown_pct         float64
usertype_Customer_pct      float64
usertype_Dependent_pct     float64
usertype_Subscriber_pct    float64
gender_Female_pct          float64
gender_Male_pct            float64
trip_count                 uint8
dtypes: float64(13), int64(2), object(3), uint8(1)
memory usage: 592.6+ MB


In [56]:
departure_joindf.to_csv('departure_joindf.csv')

## Arrival Joined Dataframe

In [57]:
dfmodel_end = dfmodel.drop(columns=['year','month','day','hour'])

In [58]:
dfmodel_end['year'] = dfmodel_end.stoptime.dt.year
dfmodel_end['month'] = dfmodel_end.stoptime.dt.month
dfmodel_end['day'] = dfmodel_end.stoptime.dt.day
dfmodel_end['hour'] = dfmodel_end.stoptime.dt.hour
dfmodel_end['num_day_of_week'] = dfmodel_end.stoptime.dt.weekday

In [59]:
dfamodel = dfmodel_end.groupby(['to_station_name','year','month','day','hour','num_day_of_week']).agg({'tripduration':'mean',
                                                                                  'temperature':'mean'}).reset_index()
dfamodel.head()

,to_station_name,year,month,day,hour,num_day_of_week,tripduration,temperature
0,2112 W Peterson Ave,2015,5,17,13,6,25.916667,77.0
1,2112 W Peterson Ave,2015,5,18,10,0,9.600000,75.0
2,2112 W Peterson Ave,2015,5,22,16,4,11.850000,62.1
3,2112 W Peterson Ave,2015,5,23,12,5,13.333333,75.9
4,2112 W Peterson Ave,2015,5,25,9,0,18.300000,69.1


In [60]:
dfamodel.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4013401 entries, 0 to 4013400
Data columns (total 8 columns):
to_station_name    4013401 non-null object
year               4013401 non-null int64
month              4013401 non-null int64
day                4013401 non-null int64
hour               4013401 non-null int64
num_day_of_week    4013401 non-null int64
tripduration       4013401 non-null float64
temperature        4013401 non-null float64
dtypes: float64(2), int64(5), object(1)
memory usage: 245.0+ MB


#### event percentages

In [61]:
dfaevents = dfmodel_end.copy()
dfaevents = dfaevents[['to_station_name','year','month','day','hour','num_day_of_week','events_cloudy','events_clear','events_not clear','events_rain or snow','events_tstorms','events_unknown']]
dfaevents.head()

,to_station_name,year,month,day,hour,num_day_of_week,events_cloudy,events_clear,events_not clear,events_rain or snow,events_tstorms,events_unknown
576426,Clarendon Ave & Junior Ter,2014,1,1,1,2,0,0,1,0,0,0
576425,Clark St & Armitage Ave,2014,1,1,1,2,0,0,1,0,0,0
576424,Clark St & Armitage Ave,2014,1,1,1,2,0,0,1,0,0,0
576423,Peoria St & Jackson Blvd,2014,1,1,1,2,0,0,1,0,0,0
576422,Peoria St & Jackson Blvd,2014,1,1,2,2,0,0,1,0,0,0


In [62]:
dfaevents['sum'] = (dfaevents['events_cloudy'] +
                   dfaevents['events_clear'] +
                   dfaevents['events_not clear'] +
                   dfaevents['events_rain or snow'] + 
                   dfaevents['events_tstorms'] +
                   dfaevents['events_unknown'])

In [63]:
dfaevents = dfaevents.sort_values(['to_station_name','year','month','day','hour','num_day_of_week'])

In [64]:
dfaevents = dfaevents.groupby(['to_station_name','year','month','day','hour','num_day_of_week']).sum().reset_index()
dfaevents.head();

In [65]:
dfaevents['events_cloudy_pct'] = (dfaevents['events_cloudy'] / dfaevents['sum'])*100
dfaevents['events_clear_pct'] = (dfaevents['events_clear'] / dfaevents['sum'])*100
dfaevents['events_not clear_pct'] = (dfaevents['events_not clear'] / dfaevents['sum'])*100
dfaevents['events_rain or snow_pct'] = (dfaevents['events_rain or snow'] / dfaevents['sum'])*100
dfaevents['events_tstorms_pct'] = (dfaevents['events_tstorms'] / dfaevents['sum'])*100
dfaevents['events_unknown_pct'] = (dfaevents['events_unknown'] / dfaevents['sum'])*100

In [66]:
dfaevents_pct = dfaevents[['to_station_name','year','month','day','hour','num_day_of_week','events_cloudy_pct','events_clear_pct','events_not clear_pct','events_rain or snow_pct','events_tstorms_pct','events_unknown_pct']]
dfaevents_pct.head()

,to_station_name,year,month,day,hour,num_day_of_week,events_cloudy_pct,events_clear_pct,events_not clear_pct,events_rain or snow_pct,events_tstorms_pct,events_unknown_pct
0,2112 W Peterson Ave,2015,5,17,13,6,100.0,0.0,0.0,0.0,0.0,0.0
1,2112 W Peterson Ave,2015,5,18,10,0,100.0,0.0,0.0,0.0,0.0,0.0
2,2112 W Peterson Ave,2015,5,22,16,4,100.0,0.0,0.0,0.0,0.0,0.0
3,2112 W Peterson Ave,2015,5,23,12,5,100.0,0.0,0.0,0.0,0.0,0.0
4,2112 W Peterson Ave,2015,5,25,9,0,0.0,0.0,0.0,100.0,0.0,0.0


#### usertype percentage

In [67]:
dfausertype = dfmodel_end.copy()
dfausertype = dfausertype[['to_station_name','year','month','day','hour','num_day_of_week','usertype_Customer','usertype_Dependent','usertype_Subscriber']]
dfausertype.head();

In [68]:
dfausertype['sum'] = (dfausertype['usertype_Customer'] +
                   dfausertype['usertype_Dependent'] +
                   dfausertype['usertype_Subscriber'])

In [69]:
dfausertype = dfausertype.sort_values(['to_station_name','year','month','day','hour','num_day_of_week'])

In [70]:
dfausertype = dfausertype.groupby(['to_station_name','year','month','day','hour','num_day_of_week']).sum().reset_index()
dfausertype.head();

In [71]:
dfausertype['usertype_Customer_pct'] = (dfausertype['usertype_Customer'] / dfausertype['sum'])*100
dfausertype['usertype_Dependent_pct'] = (dfausertype['usertype_Dependent'] / dfausertype['sum'])*100
dfausertype['usertype_Subscriber_pct'] = (dfausertype['usertype_Subscriber'] / dfausertype['sum'])*100

In [72]:
dfausertype_pct = dfausertype[['to_station_name','year','month','day','hour','num_day_of_week','usertype_Customer_pct','usertype_Dependent_pct','usertype_Subscriber_pct']]
dfausertype_pct.head()

,to_station_name,year,month,day,hour,num_day_of_week,usertype_Customer_pct,usertype_Dependent_pct,usertype_Subscriber_pct
0,2112 W Peterson Ave,2015,5,17,13,6,0.0,0.0,100.0
1,2112 W Peterson Ave,2015,5,18,10,0,0.0,0.0,100.0
2,2112 W Peterson Ave,2015,5,22,16,4,0.0,0.0,100.0
3,2112 W Peterson Ave,2015,5,23,12,5,0.0,0.0,100.0
4,2112 W Peterson Ave,2015,5,25,9,0,0.0,0.0,100.0


#### gender percentages

In [73]:
dfagender = dfmodel_end.copy()
dfagender = dfagender[['to_station_name','year','month','day','hour','num_day_of_week','gender_Female','gender_Male']]
dfagender.head();

In [74]:
dfagender['sum'] = (dfagender['gender_Female'] +dfagender['gender_Male'])

In [75]:
dfagender = dfagender.sort_values(['to_station_name','year','month','day','hour','num_day_of_week'])

In [76]:
dfagender = dfagender.groupby(['to_station_name','year','month','day','hour','num_day_of_week']).sum().reset_index()
dfagender.head();

In [77]:
dfagender['gender_Female_pct'] = (dfagender['gender_Female'] / dfagender['sum'])*100
dfagender['gender_Male_pct'] = (dfagender['gender_Male'] / dfagender['sum'])*100

In [78]:
dfagender_pct = dfagender[['to_station_name','year','month','day','hour','num_day_of_week','gender_Female_pct','gender_Male_pct','sum']]
dfagender_pct.head()

,to_station_name,year,month,day,hour,num_day_of_week,gender_Female_pct,gender_Male_pct,sum
0,2112 W Peterson Ave,2015,5,17,13,6,100.0,0.0,1
1,2112 W Peterson Ave,2015,5,18,10,0,0.0,100.0,1
2,2112 W Peterson Ave,2015,5,22,16,4,0.0,100.0,1
3,2112 W Peterson Ave,2015,5,23,12,5,0.0,100.0,1
4,2112 W Peterson Ave,2015,5,25,9,0,100.0,0.0,1


### Join

In [79]:
joindf = pd.merge(dfamodel, dfaevents_pct, how='left',on=['to_station_name','year','month','day','hour','num_day_of_week'])

In [80]:
joindf = pd.merge(joindf, dfausertype_pct, how='left', on=['to_station_name','year','month','day','hour','num_day_of_week'])

In [81]:
arrival_joindf = pd.merge(joindf, dfagender_pct, how='left', on=['to_station_name','year','month','day','hour','num_day_of_week'])

In [82]:
arrival_joindf = arrival_joindf.rename(columns={'sum':'trip_count'})

In [83]:
arrival_joindf['year'] = arrival_joindf['year'].astype(str)
arrival_joindf['month'] = arrival_joindf['month'].astype(str)
arrival_joindf['day'] = arrival_joindf['day'].astype(str)

In [84]:
arrival_joindf = arrival_joindf.set_index('to_station_name')

In [85]:
arrival_joindf.head()

,year,month,day,hour,num_day_of_week,tripduration,temperature,events_cloudy_pct,events_clear_pct,events_not clear_pct,events_rain or snow_pct,events_tstorms_pct,events_unknown_pct,usertype_Customer_pct,usertype_Dependent_pct,usertype_Subscriber_pct,gender_Female_pct,gender_Male_pct,trip_count
to_station_name,,,,,,,,,,,,,,,,,,,
2112 W Peterson Ave,2015,5,17,13,6,25.916667,77.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,1
2112 W Peterson Ave,2015,5,18,10,0,9.600000,75.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,1
2112 W Peterson Ave,2015,5,22,16,4,11.850000,62.1,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,1
2112 W Peterson Ave,2015,5,23,12,5,13.333333,75.9,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,1
2112 W Peterson Ave,2015,5,25,9,0,18.300000,69.1,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,1


In [86]:
arrival_joindf.to_csv('arrival_joindf.csv')